In [ ]:
import pandas as pd
import logging
import os
from google.cloud import bigquery
import re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from gensim import similarities
import pickle

In [ ]:
from google.colab import files 
upload = files.upload()

Saving key.json to key.json


In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key.json'
client = bigquery.Client()
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

path = "data/posts5months.pkl"
#key = 'fakeKey38i7-4259.json'

if not os.path.isdir('data/'):
    os.makedirs('data/')

# Set GOOGLE_APPLICATION_CREDENTIALS before querying

def bigQuery(QUERY):
    """ 
    Instantiates a client using a key, 
    Requests a SQL query from the Big Query API,
    Returns the queried table as a DataFrame
    """
    #client = bigquery.Client.from_service_account_json(key)
    job_config = bigquery.QueryJobConfig()
    job_config.use_legacy_sql = False
    query_job = client.query(QUERY, job_config=job_config)
    return query_job.result().to_dataframe()

# SQL query for Google BigQuery
QUERY = (
    """
    SELECT created_utc, subreddit, author, domain, url, num_comments, 
           score, title, selftext, id, gilded, retrieved_on, over_18 
    FROM   `fh-bigquery.reddit_posts.*` 
    WHERE  _table_suffix IN ( '2018_10') 
           AND LENGTH(selftext) > 500
           AND LENGTH(title) > 15
           AND LENGTH(title) < 345
           AND (score > 15 or score < -15) 
           AND is_self = true 
           AND NOT subreddit IS NULL 
           AND NOT subreddit = 'de' 
           AND NOT subreddit = 'test' 
           AND NOT subreddit = 'tr' 
           AND NOT subreddit = 'A6XHE' 
           AND NOT subreddit = 'es' 
           AND NOT subreddit = 'removalbot'
           AND NOT subreddit = 'tldr'
           AND NOT selftext LIKE '[removed]' 
           AND NOT selftext LIKE '[deleted]' 
    ;""")

df = bigQuery(QUERY)
df.to_pickle(path)

df = pd.read_pickle(path)
df.info(memory_usage='Deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173636 entries, 0 to 173635
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   created_utc   173636 non-null  int64 
 1   subreddit     173636 non-null  object
 2   author        173636 non-null  object
 3   domain        173636 non-null  object
 4   url           173636 non-null  object
 5   num_comments  173636 non-null  int64 
 6   score         173636 non-null  int64 
 7   title         173636 non-null  object
 8   selftext      173636 non-null  object
 9   id            173636 non-null  object
 10  gilded        173636 non-null  int64 
 11  retrieved_on  173636 non-null  int64 
 12  over_18       173636 non-null  bool  
dtypes: bool(1), int64(5), object(7)
memory usage: 16.1+ MB


# Let's check out the data a little bit

In [ ]:
df.head()

,created_utc,subreddit,author,domain,url,num_comments,score,title,selftext,id,gilded,retrieved_on,over_18
0,1538871394,math,lagrizle,self.math,https://www.reddit.com/r/math/comments/9m0xxu/...,15,39,Looking for a mathematical proof or any furthe...,A long time ago I was introduced to a simple t...,9m0xxu,0,1542567352,False
1,1538881895,math,FormsOverFunctions,self.math,https://www.reddit.com/r/math/comments/9m22rv/...,49,303,A question about elliptic curves,"I just stumbled upon this puzzle, which initia...",9m22rv,0,1542567879,False
2,1538887632,math,Chem_Whale2021,self.math,https://www.reddit.com/r/math/comments/9m2md1/...,13,37,Finally accepted math as my passion,Transfer College student here who after changi...,9m2md1,0,1542568127,False
3,1538895475,math,rbktech,self.math,https://www.reddit.com/r/math/comments/9m39kk/...,45,56,Stuck between two departments and don’t know h...,"I’m interested in mathematical physics, with s...",9m39kk,0,1542568436,False
4,1538939399,hardware,eric98k,self.hardware,https://www.reddit.com/r/hardware/comments/9m7...,87,87,AMD Polaris 12nm refresh coming in October and...,[PCOnline article referring to AIB manufacture...,9m7siz,0,1542570543,False


In [ ]:
len(df.author.unique())

124942

In [ ]:
len(df.subreddit.unique())

10630

In [ ]:
original_df = df

In [ ]:
df = df.drop(['domain', 'url', 'gilded', 'retrieved_on'], axis = 1)

In [ ]:
nltk.download('stopwords')
stop = stopwords.words('english')
stop.extend([x.replace("\'", "") for x in stop])
stop.extend(['nbsp', 'also', 'really', 'ive', 'even', 'jon', 'lot', 'could', 'many'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def  clean_text(df, text):
    df[text] = df[text].str.lower()
    df[text] = df[text].apply(lambda elem: 
    re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem)) 
    df[text] = df[text].apply(lambda x:
    ' '.join([word for word in x.split() if word not in (stop)]))
    df[text] = df[text].str.replace("\\n", " ")
    df[text] = df[text].str.replace(r"http\S+", " ")
    df[text] = df[text].str.replace(r"#f", "")
    df[text] = df[text].str.replace(r"[\’\'\`\":]", "")
    # remove numbers
    df[text] = df[text].apply(lambda elem: 
    re.sub(r"\d+", "", elem))
    return df


In [ ]:
df = clean_text(original_df, 'title')
df = clean_text(original_df, 'selftext')

In [ ]:
df['text'] = df.title + ' ' + df.selftext
df = df.drop(['title', 'selftext'], axis = 1)

2022-03-19 07:15:29,776 : INFO : NumExpr defaulting to 8 threads.


In [86]:


def groupPosts(x):
    # Group users' id's by post 
    return pd.Series(dict(id = ", ".join(x['id']),                    
                          text = ", ".join(x['text'])))
    
af = df.groupby('author').apply(groupPosts) 
xf = df.groupby('subreddit').apply(groupPosts) 


In [ ]:

if not os.path.isdir('model/'):
    os.makedirs('model/')

# Count vectorization for LDA
cv = CountVectorizer(token_pattern='\\w{3,}', max_df=.30, min_df=.0001, 
                     stop_words=stop, ngram_range=(1,1), lowercase=False,
                     dtype='uint8')
cv_fit = cv.fit_transform(df.text).transpose()
cvmodel = 'model/count_vec_model_full.pkl'
pickle.dump(cv_fit, open(cvmodel, 'wb'))


In [ ]:
cv_fit1 = cv.fit_transform(af.text).transpose()
cvmodel1 = 'model/cv_user.pkl'
pickle.dump(cv_fit1, open(cvmodel1, 'wb'))

cv_fit2 = cv.fit_transform(xf.text).transpose()
cvmodel2 = 'model/cv_sub.pkl'
pickle.dump(cv_fit2, open(cvmodel2, 'wb'))

In [ ]:
data = df
data = data.drop(['created_utc', 'domain', 'url', 'retrieved_on', 'gilded'], axis = 1)
data.head()

,subreddit,author,num_comments,score,id,over_18,text
0,math,lagrizle,15,39,9m0xxu,False,looking mathematical proof information problem...
1,math,FormsOverFunctions,49,303,9m22rv,False,question elliptic curves stumbled upon puzzle ...
2,math,Chem_Whale2021,13,37,9m2md1,False,finally accepted math passion transfer college...
3,math,rbktech,45,56,9m39kk,False,stuck two departments know deal im interested ...
4,hardware,eric98k,87,87,9m7siz,False,amd polaris nm refresh coming october november...


In [ ]:
from gensim import corpora, matutils, models

# Convert sparse matrix of term-doc counts to a gensim corpus
cv_corpus = matutils.Sparse2Corpus(cv_fit2)
pickle.dump(cv_corpus, open('data/cv_corpus2.pkl', 'wb'))

# Maps index to term
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

# This is for Python 3, Need this for something at the end
id2word = corpora.Dictionary.from_corpus(cv_corpus, id2word=id2word)
pickle.dump(id2word, open('data/id2word.pkl', 'wb'))


2022-03-19 07:41:37,360 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-03-19 07:41:45,126 : INFO : adding document #10000 to Dictionary(0 unique tokens: [])
2022-03-19 07:41:46,329 : INFO : built Dictionary(249495 unique tokens: ['topic', 'prep', 'conference', 'call', 'volume']...) from 10630 documents (total 20419086 corpus positions)


In [ ]:
from gensim.models.ldamulticore import LdaMulticore
passes = 100
lda = models.LdaMulticore(corpus=cv_corpus, num_topics=20, id2word=id2word, passes=passes, 
                              workers=75, random_state=42, eval_every=None, chunksize=6000)

2022-03-19 07:41:47,157 : INFO : using symmetric alpha at 0.05
2022-03-19 07:41:47,158 : INFO : using symmetric eta at 0.05
2022-03-19 07:41:47,195 : INFO : using serial LDA version on this node
2022-03-19 07:41:47,669 : INFO : running online LDA training, 20 topics, 100 passes over the supplied corpus of 10630 documents, updating every 450000 documents, evaluating every ~0 documents, iterating 50x with a convergence threshold of 0.001000
2022-03-19 07:41:47,671 : INFO : training LDA model using 75 processes
2022-03-19 07:41:50,409 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #6000/10630, outstanding queue size 1
2022-03-19 07:42:15,719 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #10630/10630, outstanding queue size 2
2022-03-19 07:42:56,686 : INFO : topic #16 (0.050): 0.002*"hes" + 0.001*"level" + 0.001*"players" + 0.001*"damage" + 0.001*"felt" + 0.001*"told" + 0.001*"gets" + 0.001*"power" + 0.001*"playing" + 0.001*"hand"
2022-03-19 07:42:56,69

In [ ]:
lda.save('data/gensim_lda.model')

2022-03-19 09:32:31,917 : INFO : saving LdaState object under data/gensim_lda.model.state, separately None
2022-03-19 09:32:32,042 : INFO : saved data/gensim_lda.model.state
2022-03-19 09:32:33,129 : INFO : saving LdaMulticore object under data/gensim_lda.model, separately ['expElogbeta', 'sstats']
2022-03-19 09:32:33,130 : INFO : storing np array 'expElogbeta' to data/gensim_lda.model.expElogbeta.npy
2022-03-19 09:32:33,144 : INFO : not storing attribute dispatcher
2022-03-19 09:32:33,144 : INFO : not storing attribute state
2022-03-19 09:32:33,145 : INFO : not storing attribute id2word
2022-03-19 09:32:33,150 : INFO : saved data/gensim_lda.model


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from gensim.models import CoherenceModel

In [ ]:
#data_lemmatized = WordNetLemmatizer(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda.log_perplexity(cv_corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=xf.text, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

2022-03-19 09:44:46,875 : INFO : -9.146 per-word bound, 566.4 perplexity estimate based on a held-out corpus of 10630 documents with 20419086 words
2022-03-19 09:44:46,935 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate probabilities from sliding windows



Perplexity:  -9.145672512776013


2022-03-19 09:44:57,646 : INFO : serializing accumulator to return to master...
2022-03-19 09:44:57,646 : INFO : serializing accumulator to return to master...
2022-03-19 09:44:57,645 : INFO : serializing accumulator to return to master...
2022-03-19 09:44:57,646 : INFO : serializing accumulator to return to master...
2022-03-19 09:44:57,646 : INFO : serializing accumulator to return to master...
2022-03-19 09:44:57,646 : INFO : serializing accumulator to return to master...
2022-03-19 09:44:57,646 : INFO : serializing accumulator to return to master...
2022-03-19 09:44:57,651 : INFO : accumulator serialized
2022-03-19 09:44:57,653 : INFO : accumulator serialized
2022-03-19 09:44:57,651 : INFO : accumulator serialized
2022-03-19 09:44:57,652 : INFO : accumulator serialized
2022-03-19 09:44:57,651 : INFO : accumulator serialized
2022-03-19 09:44:57,651 : INFO : accumulator serialized
2022-03-19 09:44:57,650 : INFO : accumulator serialized
2022-03-19 09:45:00,524 : INFO : 7 accumulators 


Coherence Score:  nan


In [ ]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[cv_corpus]

# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

In [ ]:
print(lda_docs[251])
print(xf.iloc[251])
lda.print_topics(num_words = 5, num_topics = 5)

2022-03-19 10:07:11,651 : INFO : topic #11 (0.050): 0.005*"questions" + 0.004*"job" + 0.003*"question" + 0.003*"thread" + 0.003*"company"
2022-03-19 10:07:11,656 : INFO : topic #13 (0.050): 0.004*"league" + 0.004*"esports" + 0.004*"lane" + 0.003*"team" + 0.003*"season"
2022-03-19 10:07:11,661 : INFO : topic #14 (0.050): 0.004*"told" + 0.003*"family" + 0.003*"felt" + 0.003*"talk" + 0.003*"friend"
2022-03-19 10:07:11,664 : INFO : topic #12 (0.050): 0.004*"app" + 0.004*"update" + 0.004*"phone" + 0.004*"screen" + 0.004*"games"
2022-03-19 10:07:11,668 : INFO : topic #0 (0.050): 0.009*"games" + 0.006*"cards" + 0.005*"card" + 0.005*"playing" + 0.005*"deck"


[(7, 0.10344034), (8, 0.022945836), (13, 0.024232041), (14, 0.834367), (19, 0.014037015)]
id         9npplj, 9mhrgi, 9qg5js, 9r2spv, 9n2d6q, 9rpa3k
text    experience naltrexone sinclair method posted m...
Name: Alcoholism_Medication, dtype: object


[(11,
  '0.005*"questions" + 0.004*"job" + 0.003*"question" + 0.003*"thread" + 0.003*"company"'),
 (13,
  '0.004*"league" + 0.004*"esports" + 0.004*"lane" + 0.003*"team" + 0.003*"season"'),
 (14,
  '0.004*"told" + 0.003*"family" + 0.003*"felt" + 0.003*"talk" + 0.003*"friend"'),
 (12,
  '0.004*"app" + 0.004*"update" + 0.004*"phone" + 0.004*"screen" + 0.004*"games"'),
 (0,
  '0.009*"games" + 0.006*"cards" + 0.005*"card" + 0.005*"playing" + 0.005*"deck"')]

In [ ]:
from gensim import models
import numpy as np
def loadingPickles():
  return id2word, cvmodel2, cv_corpus, lda

In [ ]:
lsi = models.LsiModel(corpus=cv_corpus, id2word=id2word, num_topics=10)
lsi_corpus = lsi[cv_corpus]

# Dump the resulting document vectors into a list
doc_vecs = [doc for doc in lsi_corpus]

2022-03-19 10:26:57,665 : INFO : using serial LSI version on this node
2022-03-19 10:26:57,666 : INFO : updating model with new documents
2022-03-19 10:26:58,851 : INFO : preparing a new chunk of documents
2022-03-19 10:27:00,365 : INFO : using 100 extra samples and 2 power iterations
2022-03-19 10:27:00,530 : INFO : 1st phase: constructing (249495, 110) action matrix
2022-03-19 10:27:02,148 : INFO : orthonormalizing (249495, 110) action matrix
2022-03-19 10:27:14,101 : INFO : 2nd phase: running dense svd on (110, 10630) matrix
2022-03-19 10:27:14,393 : INFO : computing the final decomposition
2022-03-19 10:27:14,394 : INFO : keeping 10 factors (discarding 35.047% of energy spectrum)
2022-03-19 10:27:14,520 : INFO : processed documents up to #10630
2022-03-19 10:27:14,530 : INFO : topic #0(12272.665): 0.076*"told" + 0.073*"hes" + 0.064*"friend" + 0.063*"room" + 0.062*"man" + 0.061*"guy" + 0.060*"felt" + 0.060*"talk" + 0.058*"tried" + 0.057*"job"
2022-03-19 10:27:14,537 : INFO : topic #

In [ ]:
# Create an index transformer that calculates similarity based on our space
index = similarities.MatrixSimilarity(doc_vecs, num_features=300)

# Return the sorted list of cosine similarities to the docu document
docu = 5 # Change docu as needed
sims = sorted(enumerate(index[doc_vecs[docu]]), key=lambda item: -item[1])
np.r_[sims[:10] , sims[-10:]]

2022-03-19 10:27:35,863 : INFO : creating matrix with 10630 documents and 300 features


array([[ 5.00000000e+00,  1.00000000e+00],
       [ 4.89400000e+03,  9.88597512e-01],
       [ 6.00800000e+03,  9.88025546e-01],
       [ 7.14000000e+03,  9.85120714e-01],
       [ 1.87300000e+03,  9.81784165e-01],
       [ 1.07000000e+02,  9.81573164e-01],
       [ 4.63600000e+03,  9.80565131e-01],
       [ 9.12500000e+03,  9.80550230e-01],
       [ 5.97500000e+03,  9.80254292e-01],
       [ 3.47500000e+03,  9.79951501e-01],
       [ 1.02880000e+04, -1.44712552e-02],
       [ 1.28400000e+03, -1.55248046e-02],
       [ 3.49900000e+03, -1.82465911e-02],
       [ 5.15700000e+03, -2.30116062e-02],
       [ 1.37000000e+03, -3.12310681e-02],
       [ 9.43300000e+03, -3.13631035e-02],
       [ 5.86100000e+03, -4.43916619e-02],
       [ 1.01780000e+04, -5.63704818e-02],
       [ 1.01690000e+04, -6.35192171e-02],
       [ 6.88100000e+03, -1.54954895e-01]])

In [ ]:
top = 5
for sim_doc_id, sim_score in sims[:top + 5]: 
    print("\nScore:", sim_score)
    print("Document Text:\n", xf.text[sim_doc_id])


Score: 1.0
Document Text:
  champions league pool   champions leaguepool  champions league fine tradition experimenting formats thus new one year ever eight women winners top scorers events year year place awarded premier league top scorer reducing places given  sexiest two however girls qualify one way extra place given  sexiesthere  contenders order qualified howcontender nat reason qualificationmargot robbie aus  champion championsvictoria justice usa  premier league top scorerselena gomez usa sexiest woman world awards nd sexiest awardsalexandra daddario usa  sexiest runnerupemma watson fra knockout cup champion  sexiest gal gadot isr  sexiest ana de armas cub world cup top scoreralison brie usa highest woman sept world rankings yet qualified this year eight woman split two pools top two advance semifinals oneonone standalone matchesvote upvoting downvoting pool matches last three days, world rankings sexy october  new world rankings click information live full rankings click here